# Playground

In [1]:
%load_ext autoreload
# %load_ext snakeviz
# %load_ext jupyterflame

In [2]:
import os
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)
sys.path.append(os.path.dirname(os.getcwd()))

/g/g92/izzet/projects/iopp/vani-analysis-tool/venv-quartz/bin/python
3.7.2 (default, Feb 26 2019, 08:59:10) 
[GCC 4.9.3]
sys.version_info(major=3, minor=7, micro=2, releaselevel='final', serial=0)


In [3]:
# Filter unnecessary warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
import json
import pandas as pd

metrics = {}
with open("../vani/.digio/1663953234/tmid-metrics.json", "r") as file:
    metrics = json.load(file)

metrics = pd.Series(metrics).to_numpy()

metrics[0]

[{'start': 0,
  'stop': 248265188352,
  'read': {'uniq_ranks': 1280,
   'agg_dur': 0.0395965017,
   'total_io_size': 21508390912,
   'uniq_filenames': 2,
   'bw_sum': 2075655936.0,
   'ops': 1282},
  'write': {'uniq_ranks': 1,
   'agg_dur': 0.5793977976,
   'total_io_size': 1230852944,
   'uniq_filenames': 772,
   'bw_sum': 1169473.0,
   'ops': 2303},
  'metadata': {'uniq_ranks': 1280,
   'agg_dur': 96.4753646851,
   'uniq_filenames': 1547,
   'ops': 23877}},
 {'start': 248265188352,
  'stop': 248507634825,
  'read': {'uniq_ranks': 0,
   'agg_dur': 0.0,
   'total_io_size': 0,
   'uniq_filenames': 0,
   'bw_sum': 0.0,
   'ops': 0},
  'write': {'uniq_ranks': 0,
   'agg_dur': 0.0,
   'total_io_size': 0,
   'uniq_filenames': 0,
   'bw_sum': 0.0,
   'ops': 0},
  'metadata': {'uniq_ranks': 1,
   'agg_dur': 6.2e-06,
   'uniq_filenames': 1,
   'ops': 1}}]

In [5]:
import dask.dataframe as dd
import dask_jobqueue
import logging
import socket
from dask.distributed import Client, LocalCluster

node_memory = 1600  # node memory in GB
n_workers_per_node = 16  # number of worker processes per node
n_threads_per_worker = 1
worker_time = 120  # job time per node for worker
worker_queue = "pdebug"  # queue to be used per worker
host = socket.gethostname()
dashboard_address = '{}:8788'.format(socket.gethostname())
memory = '{}GB'.format(node_memory/n_workers_per_node)
job_extra = ['-nnodes 1',
             '-G asccasc',
             '-q {}'.format(worker_queue),
             '-W {}'.format(worker_time)]
local_directory = "/var/tmp/dask2/"
cluster = LocalCluster(dashboard_address=f"{host}:7788",
                       host=host,
                        local_directory=f"{local_directory}/local",
                        memory_limit=0,
                        n_workers=n_workers_per_node,
                        silence_logs=logging.DEBUG)
client = Client(cluster, set_as_default=True)
client


distributed.http.proxy - INFO - To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at: tcp://192.168.135.138:44151
distributed.scheduler - INFO -   dashboard at:      192.168.135.138:7788
distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.135.138:46754'
distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.135.138:35220'
distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.135.138:43320'
distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.135.138:33048'
distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.135.138:40966'
distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.135.138:35673'
distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.135.138:36535'
distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.135.138:41645'
d

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://192.168.135.138:7788/status,
Dashboard: http://192.168.135.138:7788/status,Workers: 16
Total threads: 48,Total memory: 0 B
Status: running,Using processes: True
Comm: tcp://192.168.135.138:44151,Workers: 16
Dashboard: http://192.168.135.138:7788/status,Total threads: 48
Started: Just now,Total memory: 0 B
Comm: tcp://192.168.135.138:34281,Total threads: 3
Dashboard: http://192.168.135.138:42657/status,Memory: 0 B
Nanny: tcp://192.168.135.138:34426,


In [6]:
cluster.scale(n_workers_per_node*2)

In [7]:
time_min, time_max = metrics[0][0]['start'], metrics[0][0]['stop']
ops_min, ops_max = 0, 1280
io_size_min, io_size_max = 0, metrics[0][0]['read']['total_io_size'] + metrics[0][0]['write']['total_io_size']
files_min, files_max = 0, 1547


distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.135.138:37486'


In [8]:
print(time_min, time_max)
print(io_size_min, io_size_max)
print(files_min, files_max)

0 248265188352
0 22739243856
0 1547


distributed.nanny - INFO -         Start Nanny at: 'tcp://192.168.135.138:44617'


In [9]:
metrics_dds = dd.io.from_array(metrics)
metrics_dds

Dask Series Structure:
npartitions=1
0     object
10       ...
dtype: object
Dask Name: from_array, 1 tasks

In [10]:
metrics_dds.head()

distributed.worker - INFO -       Start worker at: tcp://192.168.135.138:40806
distributed.worker - INFO -          Listening to: tcp://192.168.135.138:40806
distributed.worker - INFO -          dashboard at:      192.168.135.138:42739
distributed.worker - INFO -       Start worker at: tcp://192.168.135.138:34717
distributed.worker - INFO - Waiting to connect to: tcp://192.168.135.138:44151
distributed.worker - INFO -          Listening to: tcp://192.168.135.138:34717
distributed.worker - INFO - -------------------------------------------------
distributed.worker - INFO -          dashboard at:      192.168.135.138:43840
distributed.worker - INFO -               Threads:                          3
distributed.worker - INFO - Waiting to connect to: tcp://192.168.135.138:44151
distributed.worker - INFO -       Local Directory: /var/tmp/dask2/local/dask-worker-space/worker-k044082f
distributed.worker - INFO - -------------------------------------------------
distributed.worker - INFO -   

0    [{'start': 0, 'stop': 248265188352, 'read': {'...
1    [{'start': 0, 'stop': 124132594176, 'read': {'...
2    [{'start': 0, 'stop': 62066297088, 'read': {'u...
3    [{'start': 0, 'stop': 31033148544, 'read': {'u...
4    [{'start': 0, 'stop': 15516574272, 'read': {'u...
dtype: object

In [11]:
import numpy as np
from scipy import stats

IS_INVERSED = dict(
    size=False,
    time=False,
    ops=False,
    files=True,
    bw=True,
    parallelism=True,
    xfer=True
)
IS_NORMAL = dict(
    size=False,
    time=False,
    ops=True,
    files=False,
    bw=False,
    parallelism=True,
    xfer=False
)
MAX_LABEL = 10
OBSERVATIONS = ["size", "time", "ops", "files", "bw", "parallelism", "xfer"]

root_metric = metrics[0][0]
size = root_metric['read']['total_io_size'] + root_metric['write']['total_io_size']
uniq_ranks = max(max(root_metric['read']['uniq_ranks'], root_metric['write']['uniq_ranks']), root_metric['metadata']['uniq_ranks'])

MIN_MAX = dict(
    size=(0, size),
    time=((root_metric['start'] * 1.0 / 1e7) / uniq_ranks, (root_metric['stop'] * 1.0 / 1e7) / uniq_ranks),
    ops=(0, root_metric['read']['ops'] + root_metric['write']['ops'] + root_metric['metadata']['ops']),
    files=(0, max(max(root_metric['read']['uniq_filenames'], root_metric['write']['uniq_filenames']), root_metric['metadata']['uniq_filenames'])),
    bw=(0, root_metric['read']['bw_sum'] + root_metric['write']['bw_sum']),
    parallelism=(0, uniq_ranks),
    xfer=(0, 16)
)


def compute_scores(metric):
    all_scores = {
        'start': metric['start'],
        'stop': metric['stop'],
    }
    for cat in ['read', 'write', 'metadata']:
        result = metric[cat]
        label_values = []
        scores = {}
        for observation in OBSERVATIONS:
            value = 0
            if observation == 'size' and 'total_io_size' in result:
                value = result['total_io_size']
            elif observation == 'time' and 'agg_dur' in result:
                value = result['agg_dur']
            elif observation == 'ops' and 'ops' in result:
                value = result['ops']
            elif observation == 'bw' and 'bw_sum' in result:
                value = result['bw_sum']
            elif observation == 'parallelism' and 'uniq_ranks' in result:
                value = result['uniq_ranks']
            elif observation == 'files' and 'uniq_filenames' in result:
                value = result['uniq_filenames']
            else:
                continue
            min_value, max_value = MIN_MAX[observation]
            min_max_indices = [np.finfo(float).min, np.finfo(float).max]
            indices = np.insert(min_max_indices, 1, [value])
            values = [min_value, value, max_value]
            if IS_NORMAL[observation]:
                values = [min_value, value, max_value/2]
            labels = [label for label in range(1, MAX_LABEL + 1)]
            if IS_INVERSED[observation]:
                labels = list(reversed(labels))
                indices = list(reversed(indices))
                values = list(reversed(values))
            results = pd.Series(values, index=indices).sort_values(ascending=False).astype(float)
            if IS_NORMAL[observation]:
                normalized_results = stats.norm.pdf(results, loc=max_value/2, scale=np.std(results))
                results = pd.Series(normalized_results, index=results.index)
            labeled_results = pd.cut(results, bins=MAX_LABEL, labels=labels)
            for min_max_index in min_max_indices:
                del labeled_results[min_max_index]
            label = labeled_results[labeled_results.index.values[0]]
            label_values.append(float(label))
            score = {
                'label': labeled_results[labeled_results.index.values[0]],
                'value': value
            }
            scores[observation] = score
        scores['score'] = np.sum(label_values) / len(scores) / MAX_LABEL
        all_scores[cat] = scores
    return all_scores


metric = metrics[0][0]
scores = compute_scores(metric)

scores


{'start': 0,
 'stop': 248265188352,
 'read': {'size': {'label': 10, 'value': 21508390912},
  'time': {'label': 1, 'value': 0.0395965017},
  'ops': {'label': 1, 'value': 1282},
  'files': {'label': 10, 'value': 2},
  'bw': {'label': 1, 'value': 2075655936.0},
  'parallelism': {'label': 10, 'value': 1280},
  'score': 0.55},
 'write': {'size': {'label': 1, 'value': 1230852944},
  'time': {'label': 1, 'value': 0.5793977976},
  'ops': {'label': 1, 'value': 2303},
  'files': {'label': 6, 'value': 772},
  'bw': {'label': 10, 'value': 1169473.0},
  'parallelism': {'label': 10, 'value': 1},
  'score': 0.4833333333333333},
 'metadata': {'time': {'label': 10, 'value': 96.4753646851},
  'ops': {'label': 4, 'value': 23877},
  'files': {'label': 1, 'value': 1547},
  'parallelism': {'label': 10, 'value': 1280},
  'score': 0.625}}

In [12]:
len(metrics[10])

1025

In [13]:
%%time

import dask

MAX_DEPTH = 10

# def compute_scores(metrics):
iterations = list(range(0, MAX_DEPTH + 1))
iterations.reverse()
all_tasks = [0] * (MAX_DEPTH + 1)

for i in iterations:
    tasks = []
    for t in range(2 ** i):
        tasks.append(dask.delayed(compute_scores)(metrics[i][t], dask_key_name=f"metric_{i}_{t}"))
    all_tasks[i] = tasks

all_scores = dask.compute(*all_tasks)

print(all_scores)

with open("all_scores.json", "w") as file:
    pd.Series(all_scores).to_json(file, indent=4)


distributed.core - INFO - Event loop was unresponsive in Scheduler for 8.03s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - INFO - Event loop was unresponsive in Nanny for 8.03s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - INFO - Event loop was unresponsive in Nanny for 8.03s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - INFO - Event loop was unresponsive in Nanny for 8.03s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - INFO - Event loop was unresponsive in Nanny for 8.03s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. Th

([{'start': 0, 'stop': 248265188352, 'read': {'size': {'label': 10, 'value': 21508390912}, 'time': {'label': 1, 'value': 0.0395965017}, 'ops': {'label': 1, 'value': 1282}, 'files': {'label': 10, 'value': 2}, 'bw': {'label': 1, 'value': 2075655936.0}, 'parallelism': {'label': 10, 'value': 1280}, 'score': 0.55}, 'write': {'size': {'label': 1, 'value': 1230852944}, 'time': {'label': 1, 'value': 0.5793977976}, 'ops': {'label': 1, 'value': 2303}, 'files': {'label': 6, 'value': 772}, 'bw': {'label': 10, 'value': 1169473.0}, 'parallelism': {'label': 10, 'value': 1}, 'score': 0.4833333333333333}, 'metadata': {'time': {'label': 10, 'value': 96.4753646851}, 'ops': {'label': 4, 'value': 23877}, 'files': {'label': 1, 'value': 1547}, 'parallelism': {'label': 10, 'value': 1280}, 'score': 0.625}}], [{'start': 0, 'stop': 124132594176, 'read': {'size': {'label': 5, 'value': 10737418240}, 'time': {'label': 1, 'value': 0.0199494027}, 'ops': {'label': 1, 'value': 640}, 'files': {'label': 10, 'value': 2}, 